In [1311]:
import PIL
import itertools
import pytesseract
import argparse
import cv2
import os
import pandas as pd
import numpy as np
from wand.image import Image
from wand.color import Color
import re
import pandas as pd
from string import punctuation
import sys
from statistics import mode
import operator
if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO
from pyfasttext import FastText
import pickle
from collections import OrderedDict
from extract import getModeSplits, mode
from sklearn.cluster import MeanShift, estimate_bandwidth
model = FastText('/corpora/LORELEI/fastText/lid.176.bin')

In [1312]:
model.predict_proba_single('Green', k=10)

[('en', 0.6931304869158441),
 ('de', 0.04898541251556121),
 ('es', 0.026136760232436676),
 ('pt', 0.022172690190369555),
 ('fr', 0.013090189787923609),
 ('ru', 0.012807392024569316),
 ('cs', 0.012650313032261632),
 ('it', 0.011847595759225608),
 ('he', 0.008193057293154825),
 ('fa', 0.007267200646247198)]

In [1271]:
with open('Mende/pickles/MendeManual-67.pkl', 'rb') as pickle_file:
    content = pickle.load(pickle_file)
with open('Mende/pickles/MendeManual-67_modedict.pkl', 'rb') as pickle_file_mode:
    modedict = pickle.load(pickle_file_mode)
with open('Mende/pickles/MendeManual-67_pd_hocr.pkl', 'rb') as pickle_file_mode:
    hocr = pickle.load(pickle_file_mode)

In [1272]:
content

{0: [[2, ['Bi nja we leila plo', 'Did you make drainages in~-be']],
  [2, ['tisiea ti luahun', 'tween the plots']]],
 1: [[1, ['Sia le ge bi hun- Yes, just as you instructed me']],
  [2, ['gei ia a nge', 'the last time']],
  [2, ['Ke nya sina wama ngi', "Then E'll be coming tomorrow to"]]],
 2: [[1, ['kpeteihun lo. Biya see the swamp. When trans']],
  [2, ['mbei hinma, ba ti', 'planting the rice, do not']],
  [1, ['lootoo waa gblagblaga cluster them together']],
  [2, ['bia vul lo sina', "<I suppose you'll be coming"]],
  [2, ['wa. ma bi na Ilo', 'tomorrow to see there your']]],
 4: [[2, ['Ke ba nya malelo', 'Then you will meet me']],
  [2, ['Mua lo', 'We shall see']]],
 5: [[1, ['Glossary of Tools/Useful Expressions on Farming']],
  [2, ['Benbei', 'Fishing net']],
  [2, ['Bumbui', 'Fishing Trap']],
  [2, ['Due', 'Brush']],
  [2, ['Due kpe', 'Brushing time']],
  [2, ['Felei', 'Winnowing time']],
  [2, ['Gaaga kpe', 'Time for clearing after burning']],
  [2, ['Hambui', 'An object made o

In [1293]:
def cluster1d(x):
    X = np.array(list(zip(x, np.zeros(len(x)))), dtype=np.float32)
    bandwidth = estimate_bandwidth(X, quantile=0.03)
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    ms.fit(X)
    labels = ms.labels_
    labels_unique = np.unique(labels)
    n_clusters_ = len(labels_unique)
    cluster_centers = np.argsort(ms.cluster_centers_[:,0])

    newlabelindex = {}
    for index, index2 in zip(list(labels_unique), list(cluster_centers)):
        newlabelindex[index] = index2
    
    lookup = {}
    line2members = OrderedDict()
    members2line = OrderedDict()
    lookup_val = {}
    for k in range(n_clusters_):
        my_members = labels == newlabelindex[k]
        #print("cluster {0}: {1}".format(k, X[my_members, 0]))
        members = [x for x in hocr.loc[my_members, "text"]]
        #print(members)
        #print('')
        line2members[k] = members
        lookup[k] = my_members
        lookup_val[k] = X[my_members, 0]
        joined = " ".join([x for x in members if isinstance(x, str)])
        members2line[joined] = k
    
    return lookup, line2members, members2line, lookup_val
    

In [1274]:
l_lookup, line2members, members2line, l_lookup_val = cluster1d(hocr["top"])

In [1275]:
c_lookup, col2members, members2col, c_lookup_val = cluster1d(hocr["left"])

In [1294]:
def assign_Hocr_Line(data, linedict):
    new_line2members = {}
    for para in data:
        for line in data[para]:
            tokens = " ".join(line[1]).split()
            scores = {}
            for line_hocr in linedict:
                tot = 0
                match = 0
                for tok in tokens:
                    if tok in linedict[line_hocr]:
                        match += 1
                    tot += 1
                scores[line_hocr] = match/tot
            max_line_num = max(scores.items(), key=operator.itemgetter(1))[0]
            line.append(max_line_num)
            new_line2members[max_line_num] = line[1]
    return new_line2members

line2members = assign_Hocr_Line(content, line2members)


In [1295]:
hocr.iloc[l_lookup[10],:]

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
109,4,1,1,1,11,0,1516,909,136,54,-1,NaN
110,5,1,1,1,11,1,1516,909,136,54,95,self


In [1296]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 20)

In [1297]:
def get_wLeft(word, line):
    sample = hocr.iloc[l_lookup[line],:]
    indices = sample.index[sample['text'].str.contains(word, na=False)].tolist()
    w_Left = hocr["left"].iloc[indices[0]]
    return w_Left

In [1298]:
def return_column(string):
    for members in members2line:
        if string in members:
            line = int(members2line[members])
            prevline = line - 1
    w_Left = get_wLeft(string, line)
    prev_line_sample = hocr.iloc[lookup[prevline]]
    prev_line_sample_filtered = prev_line_sample.iloc[pd.Index(prev_line_sample["text"]).notnull()]
    #print(prev_line_sample_filtered)
    returned_word_index = prev_line_sample_filtered.iloc[(prev_line_sample_filtered['left']-w_Left).abs().argsort()[:1]]
    returned_word = returned_word_index.iloc[0]["text"]
    return returned_word


In [1299]:
def matchCol(pixel_num, col_cluster_dict):
    for i in col_cluster_dict:
        if pixel_num in col_cluster_dict[i]:
            return i

In [1300]:
def determineCols(line):
    word_to_column = {}
    members = line2members[line]
    col_list = []
    for member in members:
        try:
            memb = member.split()[0]
        except:
            pass
        left = get_wLeft(memb, line)
        col = matchCol(left, c_lookup_val)
        col_list.append(col)
    return col_list

In [1301]:
def binCols(list_of_col_idxs):
    new = []
    already_added = set()
    for idx, item in enumerate(list_of_col_idxs):
        forward_match = False
        if idx < len(list_of_col_idxs)-1:
            forward_match = list_of_col_idxs[idx+1] == item + 1
        if forward_match:
            new.append([item, list_of_col_idxs[idx+1]])
            already_added.add(item)
            already_added.add(list_of_col_idxs[idx+1])
        else:
            if item not in already_added:
                new.append([item])
                already_added.add(item)
    colOrg = OrderedDict()
    table = int()
    for i, col_num in enumerate(new):
        x = i+1
        if x % 2 == 0:
            table += 1
            colOrg[table] = [new[i-1], col_num]
    print(new)
    return colOrg
        

In [1302]:
def compareAgainstMode(data):
    replacements = []
    deletions = []
    previous_para_id = data
    collapse_marker = False
    candidate_lines = []
    for i in data:
        for line_idx, c in enumerate(data[i]):
            abs_line_num = c[2]
            if c[0] != modedict[i] or c[0] % 2 != 0:
                candidate_lines.append(abs_line_num)
                line = c
                c_para = int()
                index_in_para = int()
                if line_idx != 0:
                    line_membs_cols = determineCols(data[i][line_idx][2])
                    prev_line_num = data[i][line_idx-1][2]
                    p_line = data[i][line_idx-1]
                    p_line_membs_cols = determineCols(prev_line_num)
                    index_in_para = line_idx-1
                    c_para = i
                else:
                    line_membs_cols = determineCols(data[i][line_idx][2])
                    prev_line_num = data[previous_para_id][-1][2]
                    if prev_line_num == abs_line_num - 1:
                        p_line = data[previous_para_id][-1]
                        p_line_membs_cols = determineCols(prev_line_num)
                        index_in_para = -1
                        c_para = previous_para_id
                uniques = sorted(set(line_membs_cols + p_line_membs_cols))
                col_bins = binCols(uniques)                    
                culprit_tables = {}
                for table in col_bins:
                    for idx, col in enumerate(col_bins[table]):
                        if not any(item in line_membs_cols for item in col):
                            culprit_tables.setdefault(table, [])
                            culprit_tables[table].append(idx-1)
                culprit_idxs = []
                for culprit in culprit_tables:
                    try:
                        bad_columns = culprit_tables[culprit][0]
                        for index, x in enumerate(line_membs_cols):
                            if x in col_bins[culprit][bad_columns]:
                                culprit_idxs.append((index, col_bins[culprit][bad_columns], culprit_tables[culprit][0]))
                    except:
                        pass
                
                if len(culprit_idxs) > 0:
                    for y in culprit_idxs:
                        for prev_idx, q in enumerate(p_line_membs_cols):
                            if q in y[1]:
                                prev_bad_abs = prev_idx
                        try:
                            replacement_phrase = p_line[1][prev_bad_abs] + ' ' + line[1][y[0]]
                            replacements.append((c_para, index_in_para, prev_bad_abs, replacement_phrase))
                            deletions.append((i, line_idx, y[0]))
                        except Exception as e:
                            print("replacement index error")

        previous_para_id = i

In [1303]:
compareAgainstMode(content)

KeyError: 7

In [1304]:
content

{0: [[2,
   ['Bi nja we leila plo tisiea ti luahun',
    'Did you make drainages in~-be tween the plots'],
   1,
   1]],
 1: [[2,
   ['Ke nya sina wama ngi mbei hinma, ba ti',
    "Then E'll be coming tomorrow to kpeteihun lo. Biya see the swamp. When trans planting the rice, do not lootoo waa gblagblaga cluster them together"],
   5,
   1]],
 2: [[1,
   ['lootoo waa gblagblaga cluster them together wa. ma bi na Ilo'],
   7,
   1]],
 4: [[2, ['Ke ba nya malelo', 'Then you will meet me'], 12, 1],
  [2, ['Mua lo', 'We shall see'], 13, 1]],
 5: [[2, ['Benbei', 'Fishing net'], 15, 15],
  [2, ['Bumbui', 'Fishing Trap'], 16, 16],
  [2, ['Due', 'Brush'], 17, 17],
  [2, ['Due kpe', 'Brushing time'], 18, 17],
  [2, ['Felei', 'Winnowing time'], 19, 19],
  [2, ['Gaaga kpe', 'Time for clearing after burning'], 20, 1],
  [2, ['Hambui', 'An object made of wire hung over'], 21, 21]],
 6: [[1, ['fire for drying fish or meat'], 22, 1],
  [2, ['Kali', 'Hoe'], 22, 22],
  [2, ['Kondei', 'Mortar'], 23, 23]

In [1305]:
"""
Delete headers and such
"""
def sortShorties(data):
    for i in data:
        data[i] = list(itertools.filterfalse(lambda item: item[0] == 1 and item[1][0][0].isupper(), data[i]))

sortShorties(content)

In [1306]:
"""
This function attempts to designate lines that are continuations of the previous line.
"""
def matchStrayLines(data):
    replacements = []
    deletions = []
    previous_para_id = int()
    for i in data:
        for line_idx, c in enumerate(data[i]):
            if c[0] == 1 and c[1][0][0].islower():
                in_phrases_idx = 0
                current_phrase = c[1][0]
                try:
                    source_list = model.predict_proba_single(c[1][0].lower())[0]
                except:
                    source_list = ('nothin', 0)
                if line_idx == 0:
                    previous_phrases = data[previous_para_id][-1][1]
                    try:
                        target_list = [model.predict_proba_single(y.lower())[0] for y in previous_phrases]
                    except:
                        continue
                    print(current_phrase, source_list)
                    print(previous_phrases, target_list)
                    addon = str()
                    largestlength = 0
                    try:
                        prev_word = return_column(c[1][0])
                        column_num = int()
                        for index, phrase in enumerate(previous_phrases):
                            if prev_word in phrase:
                                column_num = index
                        print(column_num)
                        replacement_phrase = previous_phrases[column_num] + ' ' + current_phrase
                        replacements.append((previous_para_id, -1, column_num, replacement_phrase))
                        deletions.append((i, 0, 0))
                        in_phrases_idx += 1
                    except:
                        for phrase, FTOutTup in zip(previous_phrases, target_list):
                            if (source_list[0] == 'en' and FTOutTup[0] == 'en') or (source_list[0] != 'en' and FTOutTup[0] != 'en'):
                                replacement_phrase = phrase + ' ' + current_phrase
                                replacements.append((previous_para_id, -1, in_phrases_idx, replacement_phrase))
                                deletions.append((i, 0, 0))
                            in_phrases_idx += 1                           
                else:
                    previous_phrases = data[i][line_idx-1][1]
                    try:
                        target_list = [model.predict_proba_single(y.lower())[0] for y in previous_phrases]
                    except:
                        continue
                    addon = str()
                    try:
                        prev_word = return_column(c[1][0])
                        column_num = int()
                        for index, phrase in enumerate(previous_phrases):
                            if prev_word in phrase:
                                column_num = index
                        replacement_phrase = previous_phrases[column_num] + ' ' + current_phrase
                        replacements.append((previous_para_id, -1, column_num, replacement_phrase))
                        deletions.append((i, 0, 0))
                        in_phrases_idx += 1
                    except:
                        for phrase, FTOutTup in zip(previous_phrases, target_list):
                            if source_list[0] == 'en' and FTOutTup[0] == 'en':
                                replacement_phrase = phrase + ' ' + current_phrase
                                replacements.append((i, line_idx-1, in_phrases_idx, replacement_phrase))
                                deletions.append((i, line_idx, 0))
                            in_phrases_idx += 1
        print('')
        previous_para_id = i
    for replacement in replacements:
        pi, li, phri, phrase = replacement[0], replacement[1], replacement[2], replacement[3]
        data[pi][li][1][phri] = phrase
    for deletion in deletions:
        pi, li, phri = deletion[0], deletion[1], deletion[2]
        print(pi, li, phri)
        try:
            del data[pi][li]
        except:
            print('already deleted')
    return replacements, deletions
        
matchStrayLines(content)



lootoo waa gblagblaga cluster them together wa. ma bi na Ilo ('en', 0.32949830360373117)
['Ke nya sina wama ngi mbei hinma, ba ti', "Then E'll be coming tomorrow to kpeteihun lo. Biya see the swamp. When trans planting the rice, do not lootoo waa gblagblaga cluster them together"] [('eo', 0.19990727066204522), ('en', 0.7047882144269162)]




climbing palm trees ('en', 0.8559684845370198)
['Kpakai typical of Mendes especially an farms', 'Three legged wooden chair used especially an farms'] [('en', 0.8564262408245333), ('en', 0.8442687941393312)]



2 0 0
7 0 0
7 0 0


([(1,
   -1,
   1,
   "Then E'll be coming tomorrow to kpeteihun lo. Biya see the swamp. When trans planting the rice, do not lootoo waa gblagblaga cluster them together lootoo waa gblagblaga cluster them together wa. ma bi na Ilo"),
  (6,
   -1,
   0,
   'Kpakai typical of Mendes especially an farms climbing palm trees'),
  (6,
   -1,
   1,
   'Three legged wooden chair used especially an farms climbing palm trees')],
 [(2, 0, 0), (7, 0, 0), (7, 0, 0)])

In [1307]:
content

{0: [[2,
   ['Bi nja we leila plo tisiea ti luahun',
    'Did you make drainages in~-be tween the plots'],
   1,
   1]],
 1: [[2,
   ['Ke nya sina wama ngi mbei hinma, ba ti',
    "Then E'll be coming tomorrow to kpeteihun lo. Biya see the swamp. When trans planting the rice, do not lootoo waa gblagblaga cluster them together lootoo waa gblagblaga cluster them together wa. ma bi na Ilo"],
   5,
   1]],
 2: [],
 4: [[2, ['Ke ba nya malelo', 'Then you will meet me'], 12, 1],
  [2, ['Mua lo', 'We shall see'], 13, 1]],
 5: [[2, ['Benbei', 'Fishing net'], 15, 15],
  [2, ['Bumbui', 'Fishing Trap'], 16, 16],
  [2, ['Due', 'Brush'], 17, 17],
  [2, ['Due kpe', 'Brushing time'], 18, 17],
  [2, ['Felei', 'Winnowing time'], 19, 19],
  [2, ['Gaaga kpe', 'Time for clearing after burning'], 20, 1],
  [2, ['Hambui', 'An object made of wire hung over'], 21, 21]],
 6: [[1, ['fire for drying fish or meat'], 22, 1],
  [2, ['Kali', 'Hoe'], 22, 22],
  [2, ['Kondei', 'Mortar'], 23, 23],
  [2, ['Koni', 'Axe']

In [1308]:
def matchStrayLines2(data):
    replacements = []
    deletions = []
    previous_para_id = int()
    collapse_marker = False
    for i in data:
        for line_idx, c in enumerate(data[i]):
            try:
                if len(c[1]) == 2 and (c[1][0][0].islower() or c[1][1][0].islower()) and len(data[previous_para_id][-1][1]) == 2:
                    in_phrases_idx = 0
                    first_phrase = c[1][0]
                    second_phrase = c[1][1]
                    if line_idx == 0:
                        previous_phrases = data[previous_para_id][-1][1]
                        addon = str()
                        for prev_phrase, add_phrase in zip(previous_phrases, c[1]):
                            replacement_phrase = add_phrase
                            replacements.append([previous_para_id, -1, in_phrases_idx, replacement_phrase])
                            deletions.append((i, 0))
                            in_phrases_idx += 1
                    else:
                        previous_phrases = data[i][line_idx-1][1]
                        addon = str()
                        for prev_phrase, add_phrase in zip(previous_phrases, c[1]):
                            #print("prev: ", prev_phrase)
                            #print("add: ",add_phrase)
                            replacement_phrase = add_phrase
                            replacements.append([i, line_idx-1, in_phrases_idx, replacement_phrase])
                            deletions.append((i, line_idx))
                            #print([i, line_idx-1, in_phrases_idx, replacement_phrase])
                            #print('')
                            in_phrases_idx += 1
                
            except:
                pass
                #print('')
        previous_para_id = i
    
    replacements = list(reversed(replacements))
    print(replacements)
    already_replaced = set()
    for repl_id, replacement in enumerate(replacements):
        pi, li, phri, phrase = replacement[0], replacement[1], replacement[2], replacement[3]
        try:
            pi_prev, li_prev, phri_prev, phrase_prev = replacements[repl_id+2][0], replacements[repl_id+2][1], replacements[repl_id+2][2], replacements[repl_id+2][3]
            if pi == pi_prev and li == li_prev+1 and phri == phri_prev:
                data[pi][li][1][phri] = data[pi][li][1][phri] + ' ' + phrase
                replacements[repl_id+2][3] = data[pi][li][1][phri]
                #already_replaced.add((pi_prev, li_prev, phri_prev))
            else:
                print("else: " + data[pi][li][1][phri] + ' ' + phrase)
                if (pi, li, phri) not in already_replaced:
                    print("yeah: " + data[pi][li][1][phri] + ' ' + phrase)
                    data[pi][li][1][phri] = data[pi][li][1][phri] + ' ' + phrase
        except:
            print(data[pi][li][1][phri] + ' ' + phrase)
            data[pi][li][1][phri] = data[pi][li][1][phri] + ' ' + phrase
            print('list index out of range')
    
    print('already replaced')
    print(already_replaced)
    print('')
    print("data")
    for h in data:
        print(h)
        for index, j in enumerate(data[h]):
            print("\t", index, "\t", j)
    print('')
    print('replacements')
    print(replacements)
    
    deletions = reversed(list(OrderedDict.fromkeys(deletions)))
    for deletion in deletions:
        pi, li = deletion[0], deletion[1]
        print(pi, li, phri)
        try:
            del data[pi][li]
        except:
            print('already deleted')
    return replacements, deletions
    

        
matchStrayLines2(content)

[]
already replaced
set()

data
0
	 0 	 [2, ['Bi nja we leila plo tisiea ti luahun', 'Did you make drainages in~-be tween the plots'], 1, 1]
1
	 0 	 [2, ['Ke nya sina wama ngi mbei hinma, ba ti', "Then E'll be coming tomorrow to kpeteihun lo. Biya see the swamp. When trans planting the rice, do not lootoo waa gblagblaga cluster them together lootoo waa gblagblaga cluster them together wa. ma bi na Ilo"], 5, 1]
2
4
	 0 	 [2, ['Ke ba nya malelo', 'Then you will meet me'], 12, 1]
	 1 	 [2, ['Mua lo', 'We shall see'], 13, 1]
5
	 0 	 [2, ['Benbei', 'Fishing net'], 15, 15]
	 1 	 [2, ['Bumbui', 'Fishing Trap'], 16, 16]
	 2 	 [2, ['Due', 'Brush'], 17, 17]
	 3 	 [2, ['Due kpe', 'Brushing time'], 18, 17]
	 4 	 [2, ['Felei', 'Winnowing time'], 19, 19]
	 5 	 [2, ['Gaaga kpe', 'Time for clearing after burning'], 20, 1]
	 6 	 [2, ['Hambui', 'An object made of wire hung over'], 21, 21]
6
	 0 	 [1, ['fire for drying fish or meat'], 22, 1]
	 1 	 [2, ['Kali', 'Hoe'], 22, 22]
	 2 	 [2, ['Kondei', 'Mortar

([], <list_reverseiterator at 0x7f5e114f86d8>)

In [1309]:
def deleteBlanks(data):
    for k, v in list(data.items()):
        if v == []:
            del data[k]

In [1310]:
content

{0: [[2,
   ['Bi nja we leila plo tisiea ti luahun',
    'Did you make drainages in~-be tween the plots'],
   1,
   1]],
 1: [[2,
   ['Ke nya sina wama ngi mbei hinma, ba ti',
    "Then E'll be coming tomorrow to kpeteihun lo. Biya see the swamp. When trans planting the rice, do not lootoo waa gblagblaga cluster them together lootoo waa gblagblaga cluster them together wa. ma bi na Ilo"],
   5,
   1]],
 2: [],
 4: [[2, ['Ke ba nya malelo', 'Then you will meet me'], 12, 1],
  [2, ['Mua lo', 'We shall see'], 13, 1]],
 5: [[2, ['Benbei', 'Fishing net'], 15, 15],
  [2, ['Bumbui', 'Fishing Trap'], 16, 16],
  [2, ['Due', 'Brush'], 17, 17],
  [2, ['Due kpe', 'Brushing time'], 18, 17],
  [2, ['Felei', 'Winnowing time'], 19, 19],
  [2, ['Gaaga kpe', 'Time for clearing after burning'], 20, 1],
  [2, ['Hambui', 'An object made of wire hung over'], 21, 21]],
 6: [[1, ['fire for drying fish or meat'], 22, 1],
  [2, ['Kali', 'Hoe'], 22, 22],
  [2, ['Kondei', 'Mortar'], 23, 23],
  [2, ['Koni', 'Axe']

In [1262]:
deleteBlanks(content)

In [1263]:
dictionary = {}

In [1264]:
columns = []
if content[4] != []:
    mode = modedict[4]
    for x in content[4]:
        linenum = x[0]
        splits = x[1]
        if linenum % 2 == 0:
            for idx, split in enumerate(splits):
                try:
                    columns[idx].append(split)
                    print(idx, split)
                except:
                    columns.append([])
                    columns[idx].append(split)
                    print(idx, split)
            print(linenum, splits)
        else:
            pass
    print('')
    print('Here are the columns')
    try:
        it = iter(columns)
        print(columns)
        the_len = len(columns[0])
        print('the leng', the_len)
        column_lang = {}

        for col_i, column in enumerate(columns):
            proc_col = [''.join(x.lower().split('/')) for x in column]
            FTout = model.predict_proba_single(" ".join(proc_col), k=1)
            try:
                tuple = FTout[0]
                column_lang[col_i] = tuple[0]
                if tuple[0] == 'en' and tuple[1] < .35:
                    column_lang[col_i] = 'nothin'
            except:
                column_lang[col_i] = 'nothin'
        column_index = 0
        print(column_lang)
        print(len(columns))
        while column_index < len(columns):
            try:
                print(column_index)
                print(column_lang[column_index], column_lang[column_index+1])
                if column_lang[column_index] == 'en' and column_lang[column_index + 1] != 'en':
                    for phrase1, phrase2 in zip(columns[column_index], columns[column_index+1]):
                        dictionary[phrase1] = phrase2
                        print(column_index, phrase1, phrase2)
                    column_index += 2
                elif column_lang[column_index] != 'en' and column_lang[column_index + 1] == 'en':
                    for phrase1, phrase2 in zip(columns[column_index], columns[column_index+1]):
                        dictionary[phrase2] = phrase1
                        print(column_index, phrase1, phrase2)
                    column_index += 2
                else:
                    column_index += 2
            except:
                break
        else:
            print('game cancelled')
    except:
        pass

0 Ke ba nya malelo
1 Then you will meet me
2 ['Ke ba nya malelo', 'Then you will meet me']
0 Mua lo
1 We shall see
2 ['Mua lo', 'We shall see']

Here are the columns
[['Ke ba nya malelo', 'Mua lo'], ['Then you will meet me', 'We shall see']]
the leng 2
{0: 'es', 1: 'en'}
2
0
es en
0 Ke ba nya malelo Then you will meet me
0 Mua lo We shall see
game cancelled


In [1265]:
for i in dictionary:
    print(i, '\t', '\t', dictionary[i])

Then you will meet me 	 	 Ke ba nya malelo
We shall see 	 	 Mua lo


In [1266]:
dictionary = {}

In [1267]:
modedict = getModeSplits(content)
for i in content:
    columns = []
    if content[i] != []:
        for x in content[i]:
            linenum = x[0]
            splits = x[1]
            for idx, split in enumerate(splits):
                try:
                    columns[idx].append(split)
                except:
                    columns.append([])
                    columns[idx].append(split)
            #print(linenum, splits)
        print('')
        it = iter(columns)
        the_len = len(columns[0])
        print('the leng', the_len)
        #if all(len(x) == the_len for x in columns):
        column_lang = {}
        for col_i, column in enumerate(columns):
            proc_col = [''.join(x.split('/')) for x in column]
            FTout = model.predict_proba_single(" ".join(proc_col), k=1)
            try:
                tuple = FTout[0]
                column_lang[col_i] = tuple[0]
                if tuple[0] == 'en' and tuple[1] < .35:
                    column_lang[col_i] = 'nothin'
            except:
                column_lang[col_i] = 'nothin'
        column_index = 0
        while column_index < len(columns):
            try:
                if column_lang[column_index] == 'en' and column_lang[column_index + 1] != 'en':
                    for phrase1, phrase2 in zip(columns[column_index], columns[column_index+1]):
                        dictionary[phrase1] = phrase2
                        print(column_index, phrase1, phrase2)
                    column_index += 2
                elif column_lang[column_index] != 'en' and column_lang[column_index + 1] == 'en':
                    for phrase1, phrase2 in zip(columns[column_index], columns[column_index+1]):
                        dictionary[phrase2] = phrase1
                        print(column_index, phrase1, phrase2)
                    column_index += 2
                else:
                    column_index += 2
            except:
                break
for i in dictionary:
    print(i, '\t', '\t', dictionary[i])


the leng 1
0 Bi nja we leila plo tisiea ti luahun Did you make drainages in~-be tween the plots

the leng 1
0 Ke nya sina wama ngi mbei hinma, ba ti Then E'll be coming tomorrow to kpeteihun lo. Biya see the swamp. When trans planting the rice, do not lootoo waa gblagblaga cluster them together

the leng 1

the leng 2
0 Ke ba nya malelo Then you will meet me
0 Mua lo We shall see

the leng 7

the leng 6

the leng 8

the leng 6
0 Mbogbeili Cutlass
0 Ps bogbel Cutlass for cutting down trees
0 Ndogboi Bush
0 Ndoli Hook
0 Ngombui Fire
0 Piyei Container for putting

the leng 1
Did you make drainages in~-be tween the plots 	 	 Bi nja we leila plo tisiea ti luahun
Then E'll be coming tomorrow to kpeteihun lo. Biya see the swamp. When trans planting the rice, do not lootoo waa gblagblaga cluster them together 	 	 Ke nya sina wama ngi mbei hinma, ba ti
Then you will meet me 	 	 Ke ba nya malelo
We shall see 	 	 Mua lo
Cutlass 	 	 Mbogbeili
Cutlass for cutting down trees 	 	 Ps bogbel
Bush 	 	 

In [1268]:
dictionary

{'Did you make drainages in~-be tween the plots': 'Bi nja we leila plo tisiea ti luahun',
 "Then E'll be coming tomorrow to kpeteihun lo. Biya see the swamp. When trans planting the rice, do not lootoo waa gblagblaga cluster them together": 'Ke nya sina wama ngi mbei hinma, ba ti',
 'Then you will meet me': 'Ke ba nya malelo',
 'We shall see': 'Mua lo',
 'Cutlass': 'Mbogbeili',
 'Cutlass for cutting down trees': 'Ps bogbel',
 'Bush': 'Ndogboi',
 'Hook': 'Ndoli',
 'Fire': 'Ngombui',
 'Container for putting': 'Piyei'}